Distan Domain
Gender classifieng

In [1]:
import numpy as np
from keras import applications, optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import pickle
from IPython.display import clear_output
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [32]:
blocks=[18, 14, 10, 6, 3, 0]
import numpy as np
datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = datagen.flow_from_directory(
        #'tmp/test',
        'GENDER-COLOR-FERET/test',
        target_size=(150, 150),
        batch_size=1)
def getTestModel(weightFile):
    
    
    model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    
    top_model = Sequential()
    top_model.add(Flatten(input_shape=model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(1, activation='sigmoid'))

    # add the model on top of the convolutional base
    test_model = Model(inputs=model.input, outputs=top_model(model.output))#model.add(top_model)
    if(weightFile is not None):    
        test_model.load_weights(weightFile)
    else: 
        print('Using original weights from IMAGENET')
    test_model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),metrics=['accuracy'])
    
    return test_model





def getResults(vgg16Weights,augumentation, i):    
    aug=''
    wei=''
    model=''
    if (augumentation):
        aug='Augumentation'
    else:
        aug='NoAugumentation'
        
    if vgg16Weights:
        wei='Weights'
    else:
        wei='NoWeights'
    model='vgg16_'+wei+'_'+aug+'_trainedLast_'+str(18-i)+'_Layers_.h5'
    print('Use Model: '+model)
    trainedModel=getTestModel(model) 
    prediction=trainedModel.predict_generator(test_generator,100)
    print('test accuracy: '+str(np.sum(prediction)/len(prediction)))
    
  
    

Found 235 images belonging to 2 classes.


In [38]:
# test with imagenet weights
trainedModel=getTestModel(None) 
prediction=trainedModel.predict_generator(test_generator,100)
print('test accuracy: '+str(np.sum(prediction)/len(prediction)))

Using original weights from IMAGENET
test accuracy: 0.122788209915


In [39]:
#test with weight from (trained) distant domain cats/dogs 
for i in blocks: 
    getResults(True, True,i);
    getResults(True, False,i);
    getResults(False, True,i);
    getResults(False, False,i);
    print('------------------------------------------------------------------------------')


Use Model: vgg16_Weights_Augumentation_trainedLast_0_Layers_.h5
test accuracy: 0.868209075928
Use Model: vgg16_Weights_NoAugumentation_trainedLast_0_Layers_.h5
test accuracy: 0.713978652954
Use Model: vgg16_NoWeights_Augumentation_trainedLast_0_Layers_.h5
test accuracy: 0.795077972412
Use Model: vgg16_NoWeights_NoAugumentation_trainedLast_0_Layers_.h5
test accuracy: 0.874945144653
------------------------------------------------------------------------------
Use Model: vgg16_Weights_Augumentation_trainedLast_4_Layers_.h5
test accuracy: 0.81625
Use Model: vgg16_Weights_NoAugumentation_trainedLast_4_Layers_.h5
test accuracy: 0.840189208984
Use Model: vgg16_NoWeights_Augumentation_trainedLast_4_Layers_.h5
test accuracy: 0.930809173584
Use Model: vgg16_NoWeights_NoAugumentation_trainedLast_4_Layers_.h5
test accuracy: 0.93735496521
------------------------------------------------------------------------------
Use Model: vgg16_Weights_Augumentation_trainedLast_8_Layers_.h5
test accuracy: 0.9

ResourceExhaustedError: OOM when allocating tensor with shape[1,64,150,150]
	 [[Node: block1_conv1_86/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_input_87_0_1/_25325, block1_conv1_86/kernel/read)]]
	 [[Node: sequential_87/dense_174/Sigmoid/_25333 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_186_sequential_87/dense_174/Sigmoid", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'block1_conv1_86/convolution', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launc